<a href="https://colab.research.google.com/github/vladgap/LLC/blob/main/Rotem_LLC_Washing_Simulation_OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"

from scipy import optimize

In [2]:
!pip install MLNN
from MLNN import*

  Preparing metadata (setup.py) ... done
  Created wheel for MLNN: filename=MLNN-2.2-py3-none-any.whl size=7996 sha256=d7b9b24a0e89c3163dd00686317aaab4cdf3958f7dd34f98bdde9cef91fd453f
  Stored in directory: /root/.cache/pip/wheels/a4/68/4c/c1aa01095c9ecd1cd63981432b3dd200fe7452e1e15dd6c469
Successfully built MLNN


In [3]:
!wget -O LLC.py "https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_2.py"
import LLC

--2025-03-13 09:30:44--  https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54639 (53K) [text/plain]
Saving to: ‘LLC.py’

LLC.py              100%[===================>]  53.36K  --.-KB/s    in 0.02s   

2025-03-13 09:30:45 (2.94 MB/s) - ‘LLC.py’ saved [54639/54639]



In [4]:
def CopyPasteToArray(a):
  if a.startswith('\n'):
    b=a[1:]
  else:
    b=a
  if b.endswith('\n'):
    b=b[:-1]
  c=b.replace('\t',',')
  d=c.split('\n')
  f=[]
  for e in d:
    if e.replace(',','').replace('.','').isdigit(): # only digits no letters
      f.append(list(eval(e)))
    else:
      f.append(e.split(','))
  return f

## EQUIL_rotem_washing
ניסויי הפרדה עם ממס מטוהר

In [5]:
Xscale=[21.97576471,  0.05575882], [15.2853469,   0.05047379]
Tscale=[1.89941176, 11.19176471,  1.4,         0.02921412], [2.40216868, 1.27085536, 0.17875386, 0.01997862]

scaler_X3 = preprocessing.StandardScaler().fit([[0,0]])
scaler_X3_2 = preprocessing.StandardScaler().fit([[0,0]]) # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_1 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_2 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_3 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_4 = preprocessing.StandardScaler().fit([[0]])

scaler_X3.mean_, scaler_X3.scale_ = Xscale[0], Xscale[1]
scaler_X3_2.mean_, scaler_X3_2.scale_ = [27.03608333,  0.0474    ], [14.68499343,  0.04213688] # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_1.mean_, scaler_T3_1.scale_=[Tscale[0][0]], [Tscale[1][0]]
# scaler_T3_2.mean_, scaler_T3_2.scale_=[Tscale[0][1]], [Tscale[1][1]]
scaler_T3_2.mean_, scaler_T3_2.scale_=[10.81], [1.3091664] # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_3.mean_, scaler_T3_3.scale_=[Tscale[0][2]], [Tscale[1][2]]
scaler_T3_4.mean_, scaler_T3_4.scale_=[Tscale[0][3]], [Tscale[1][3]]

In [6]:
# @title class EQUIL_rotem_washing1
class EQUIL_rotem_washing1: # ניסויי הפרדה עם ממס מטוהר
# מודל מקורי
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]])))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        predics3_2=scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]]))) # מים בקלה חושבו רק על נתונים "טובים

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]])))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]])))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

In [7]:
# @title class EQUIL_rotem_washing2
class EQUIL_rotem_washing2: # ניסויי הפרדה עם ממס מטוהר
# מודל עם מים בקלה 12% גורף
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=abs(scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]]))))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        # predics3_2=abs(scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]])))) # מים בקלה חושבו רק על נתונים "טובים
        predics3_2=np.ones([len(x),1])*12 #  תוצאות אנליטיות לא מדויקות. לקחתי 12 באופן גורף

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=abs(scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]]))))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=abs(scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]]))))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

In [8]:
# @title class EQUIL_rotem_washing3
class EQUIL_rotem_washing3: # ניסויי הפרדה עם ממס מטוהר
# מודל עם מים בקלה 12% גורף
# שינוי במשקלים של ממס בכבדה. שיהיה יותר אופקי
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=abs(scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]]))))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        # predics3_2=abs(scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]])))) # מים בקלה חושבו רק על נתונים "טובים
        predics3_2=np.ones([len(x),1])*12 #  תוצאות אנליטיות לא מדויקות. לקחתי 12 באופן גורף

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu') # modified solvent_aq weights
        manual_weights3_3=[[[-7.637254337641781, -0.490322880885377],
  [-1.8058063735873857, -0.10159866035465558]],
 [[-0.843126492292387, -2.3299686091075773]],
 [[0.2643116291427324], [0.4479958576488306]],
 [[-0.7056420231884761]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=abs(scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]]))))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=abs(scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]]))))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

# Battery simulation

In [9]:
A=np.array([[2,-3],[1,-1]])
B=np.array([1,2])
x1,x2=np.linalg.solve(A,B)
x1

5.0

In [10]:
# @title class Optimal_Washing_Battery
class Optimal_Washing_Battery:
    '''
    A class representing an optimal washing battery for a chemical plant process.
    It calculates and optimizes various parameters related to the washing process.
    '''
    def __init__(self, plant_data, stages_num=6):
        '''
        Initialize the Optimal_Washing_Battery with plant data.
        Performs initial calculations, sets up the plant input data, and runs optimizations.

        Parameters:
        plant_data (list): Contains PS_P2O5, WS_P2O5, W_flow, Prod_flow, Prod_P2O5 values
        stages_num (int): Number of stages in the battery (default is 6)
        '''

        self.stages_num=stages_num
        self.plant_data = plant_data
        PS_P2O5, PS_SO4, WS_P2O5, W_flow, W_P2O5, W_SO4, Prod_flow, Prod_P2O5 = self.plant_data
        slope_process=(PS_P2O5-WS_P2O5)/(Prod_P2O5-W_P2O5)
        # PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
        # WS_flow=PS_flow+W_flow-Prod_flow

        A=np.array([[PS_P2O5,-WS_P2O5],[1,-1]])
        b=np.array([Prod_flow*Prod_P2O5-W_flow*W_P2O5,Prod_flow-W_flow])
        PS_flow, WS_flow=np.linalg.solve(A,b)

        P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
        self.plant_input_data = PS_flow, W_flow, PS_P2O5, W_P2O5, PS_SO4, W_SO4
        # print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

        plant_data_table=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",stages_num,'---', '---'],
                                index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                                columns=['Plant data'])

        print ('Calculating entrainment = f(slope,eff) parameters')
        C=self.__calculate_C()

        print ('Calculating optimal eff and entrain')
        optimal_params=self.__calculate_optimal_params(C, slope_process)
        opt_eff, opt_entrain = optimal_params

        print ('Calculating entrainment optimal battery simulation')
        optimal_bat=LLC.Battery(stages_num=self.stages_num, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,W_P2O5,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=opt_eff, convergence=1E-3, entrainment_perc_out=[opt_entrain,0])
        self.bat=optimal_bat
        print ('Ready')

        self.summary_table=self.__prepare_summary_table(optimal_bat, plant_data_table)
        # self.summary_table

        self.profile_table=LLC.BatteryTable(optimal_bat, show=False).data
        self.full_profile_table=LLC.BatteryTableFull(optimal_bat, show=False).data

    def __calculate_C(self,entr_initial=3.6, eff_init=0.77):
        '''
        Calculate parameters for the relationship between slope, entrainment, and efficiency.
        This method runs simulations with different entrainment and efficiency values to find
        coefficients for the equation: slope = a + b * entr + c * eff + d * eff^2

        Parameters:
        entr_initial (float): Initial entrainment value
        eff_init (float): Initial efficiency value

        Returns:
        numpy.ndarray: Coefficients [a, b, c, d] for the slope equation

        נדרש להתאים את המודל לנתוני התהליך. סחיפות ויעילות הם נתוני כניסה למודל, חייבים למצוא אותם ע"י נצבה והרצה. הדרך ישרה היא הרצת אופטימיזציה עם שני משתנים וחישוב שגיאה יחסית לנתוני תהליך. ההתכנסות איטית ולא מדוייקת.

        נמצא לפני זה שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
        מרובע. מחשבים פרמטרים של פונקציה זאת

        slope = a + b * entr + c * eff + d * eff^2 ==> entr = (slope - a - c * eff - d * eff^2)/ b

        בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

        מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

        entrain=(slope_proc - C[0] - C[2] * eff - C[3] * eff**2)/C[1]

        האופטימיזציה הופכת לאחת עם משתנה בודד.
        '''

        PS_flow, W_flow, PS_P2O5, W_P2O5, PS_SO4, W_SO4 = self.plant_input_data
        slopes=[]
        # entr_initial, eff_init = 3.6, 0.77
        for entr in [entr_initial, entr_initial*.75]: # לינארי -- מספיק 2
          for eff in [eff_init,eff_init*.85,eff_init*1.1]: # פורבולה -- צריך 3
            bat=LLC.Battery(stages_num=self.stages_num, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,W_P2O5,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
            slope=(bat.yin_list[0][1]-bat.yout_list[-1][1])/(bat.xout_list[0][1]-bat.xin_list[-1][1])
            slopes.append([entr, eff, slope])
            # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
            print ('.', end=' ')
        print ('\n')
        arr=np.array(slopes)
        arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]] # entrainment approximated to porabola
        rows=[0,1,2,3] # 4 rows for 4 unknowns
        A=arr_mod[rows,0:4]
        B=arr_mod[rows,4]
        C=np.dot(np.linalg.inv(A),B) # פתרון מערכת משוואות.  מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות
        return C

    def __calculate_optimal_params(self, C, slope_process):
        '''
        Calculate optimal efficiency and entrainment parameters.
        Uses optimization to find the best efficiency value, then calculates the corresponding entrainment.

        Parameters:
        C (numpy.ndarray): Coefficients from __calculate_C method
        slope_process (float): Slope of the process line

        Returns:
        list: Optimal efficiency and entrainment values
        '''

        bounds=[(0.5,1)]
        sol=optimize.direct(self.__P2O5_calc, bounds=bounds, args=(C, slope_process), f_min=0, f_min_rtol=0.001, maxiter=100, locally_biased=False)
        eff=sol.x[0]
        entrain=(slope_process-C[0]-C[2]*eff-C[3]*eff**2)/C[1]
        print ('\n', 'optimal data:\n', 'eff:', sol.x[0], 'entrain:', entrain, 'convergence:', sol.fun)
        return [eff, entrain]

    def __P2O5_calc(self, x, C, slope_process):
        '''
        Objective function for optimization.
        Calculates the error between the model's P2O5 output and the actual plant data.

        Parameters:
        x (list): Contains the efficiency value
        C (numpy.ndarray): Coefficients from __calculate_C method
        slope_process (float): Slope of the process line

        Returns:
        float: Absolute error between model and plant P2O5 values
        '''

        PS_flow, W_flow, PS_P2O5, W_P2O5, PS_SO4, W_SO4 = self.plant_input_data
        eff = x[0]
        entrain=(slope_process-C[0]-C[2]*eff-C[3]*eff**2)/C[1]
        bat=LLC.Battery(stages_num=self.stages_num, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,W_P2O5,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=eff, convergence=1E-3, entrainment_perc_out=[entrain,0])
        error=abs((Prod_P2O5-bat.xout_list[0][1]/98*71)/Prod_P2O5)
        # print (eff, entrain, bat.yout_list[-1][1]/98*71, bat.xout_list[0][1]/98*71, error, bat.runs, ',')
        print ('.', end=' ')
        return error

    def __prepare_summary_table(self, bat, plant_data_table):
        '''
        Prepare a summary table comparing plant data with model data.

        Parameters:
        bat (LLC.Battery): Optimized battery object
        plant_data_table (pandas.DataFrame): Table containing plant data

        Returns:
        pandas.DataFrame: Summary table with plant and model data
        '''

        summary_table = plant_data_table.copy()
        summary_table['Model data']=[bat.Oin_list[0],
                                        bat.yin_list[0][1]/98*71,
                                        bat.Oout_list[-1],bat.yout_list[-1][1]/98*71,
                                        bat.Ain_list[-1],
                                        bat.Aout_list[0],
                                        bat.xout_list[0][1]/98*71,
                                        f"{bat.Aout_list[0]*bat.xout_list[0][1]/(bat.Oin_list[0]*bat.yin_list[0][1]):.1%}",
                                        f"{bat.stages_num:.0f}",
                                        bat.eff_list[0,1],
                                        str(bat.EQUIL)[17:-2]]
        return summary_table

In [11]:
# @title class Optimal_Washing_CounterCurrentBattery
class Optimal_Washing_CounterCurrentBattery:
    '''
    A class representing an optimal washing battery for a chemical plant process.
    It calculates and optimizes various parameters related to the washing process.
    '''
    def __init__(self, plant_data, stages_num=6, **kwargs):
        '''
        Initialize the Optimal_Washing_CounterCurrentBattery with plant data.
        Performs initial calculations, sets up the plant input data, and runs optimizations.

        Parameters:
        plant_data (list): Contains PS_P2O5, WS_P2O5, W_flow, Prod_flow, Prod_P2O5 values
        stages_num (int): Number of stages in the battery (default is 6)
        '''

        self.stages_num=stages_num
        self.plant_data = plant_data
        PS_P2O5, PS_SO4, WS_P2O5, W_flow, W_P2O5, W_SO4, Prod_flow, Prod_P2O5 = self.plant_data
        slope_process=(PS_P2O5-WS_P2O5)/(Prod_P2O5-W_P2O5)
        print ('slope_process:', slope_process)
        # PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
        # WS_flow=PS_flow+W_flow-Prod_flow

        A=np.array([[PS_P2O5,-WS_P2O5],[1,-1]])
        b=np.array([Prod_flow*Prod_P2O5-W_flow*W_P2O5,Prod_flow-W_flow])
        PS_flow, WS_flow=np.linalg.solve(A,b)

        P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5+W_flow*W_P2O5)
        self.plant_input_data = PS_flow, W_flow, PS_P2O5, W_P2O5, PS_SO4, W_SO4
        # print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

        plant_data_table=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",stages_num,'---', '---'],
                                index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                                columns=['Plant data'])

        print ('Calculating entrainment = f(slope,eff) parameters')
        C=self.__calculate_C(entr_initial=1, eff_init=0.86)

        print ('Calculating optimal eff and entrain')
        optimal_params=self.__calculate_optimal_params(C, slope_process)
        opt_eff, opt_entrain = optimal_params

        print ('Calculating entrainment optimal battery simulation')
        optimal_bat=LLC.CounterCurrentBattery(stages_num=self.stages_num, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,W_P2O5/71*98,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=opt_eff, entrainment_perc_out=[opt_entrain,0], **kwargs)
        self.bat=optimal_bat
        print ('Ready')

        self.summary_table=self.__prepare_summary_table(optimal_bat, plant_data_table)
        # self.summary_table

        self.profile_table=LLC.BatteryTable(optimal_bat, show=False).data
        self.full_profile_table=LLC.BatteryTableFull(optimal_bat, show=False).data

    def __calculate_C(self,entr_initial=3.6, eff_init=0.77):
        '''
        Calculate parameters for the relationship between slope, entrainment, and efficiency.
        This method runs simulations with different entrainment and efficiency values to find
        coefficients for the equation: slope = a + b * entr + c * eff + d * eff^2

        Parameters:
        entr_initial (float): Initial entrainment value
        eff_init (float): Initial efficiency value

        Returns:
        numpy.ndarray: Coefficients [a, b, c, d] for the slope equation

        נדרש להתאים את המודל לנתוני התהליך. סחיפות ויעילות הם נתוני כניסה למודל, חייבים למצוא אותם ע"י נצבה והרצה. הדרך ישרה היא הרצת אופטימיזציה עם שני משתנים וחישוב שגיאה יחסית לנתוני תהליך. ההתכנסות איטית ולא מדוייקת.

        נמצא לפני זה שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
        מרובע. מחשבים פרמטרים של פונקציה זאת

        slope = a + b * entr + c * eff + d * eff^2 ==> entr = (slope - a - c * eff - d * eff^2)/ b

        בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

        מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

        entrain=(slope_proc - C[0] - C[2] * eff - C[3] * eff**2)/C[1]

        האופטימיזציה הופכת לאחת עם משתנה בודד.
        '''

        PS_flow, W_flow, PS_P2O5, W_P2O5, PS_SO4, W_SO4 = self.plant_input_data
        slopes=[]
        # entr_initial, eff_init = 3.6, 0.77
        for entr in [entr_initial, entr_initial*.75]: # לינארי -- מספיק 2
          for eff in [eff_init,eff_init*.85,eff_init*1.1]: # פורבולה -- צריך 3
            bat=LLC.Battery(stages_num=self.stages_num, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,W_P2O5/71*98,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
            slope=(bat.yin_list[0][1]-bat.yout_list[-1][1])/(bat.xout_list[0][1]-bat.xin_list[-1][1])
            slopes.append([entr, eff, slope])
            # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
            print ('.', end=' ')
        print ('\n')
        arr=np.array(slopes)
        arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]] # entrainment approximated to porabola
        rows=[0,1,2,3] # 4 rows for 4 unknowns
        A=arr_mod[rows,0:4]
        B=arr_mod[rows,4]
        C=np.dot(np.linalg.inv(A),B) # פתרון מערכת משוואות.  מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות
        return C

    def __calculate_optimal_params(self, C, slope_process):
        '''
        Calculate optimal efficiency and entrainment parameters.
        Uses optimization to find the best efficiency value, then calculates the corresponding entrainment.

        Parameters:
        C (numpy.ndarray): Coefficients from __calculate_C method
        slope_process (float): Slope of the process line

        Returns:
        list: Optimal efficiency and entrainment values
        '''

        bounds=[(0.5,1)]
        sol=optimize.direct(self.__P2O5_calc, bounds=bounds, args=(C, slope_process), f_min=0, f_min_rtol=0.001, maxiter=100, locally_biased=False)
        eff=sol.x[0]
        entrain=(slope_process-C[0]-C[2]*eff-C[3]*eff**2)/C[1]
        print ('\n', 'optimal data:\n', 'eff:', sol.x[0], 'entrain:', entrain, 'convergence:', sol.fun)
        return [eff, entrain]

    def __P2O5_calc(self, x, C, slope_process):
        '''
        Objective function for optimization.
        Calculates the error between the model's P2O5 output and the actual plant data.

        Parameters:
        x (list): Contains the efficiency value
        C (numpy.ndarray): Coefficients from __calculate_C method
        slope_process (float): Slope of the process line

        Returns:
        float: Absolute error between model and plant P2O5 values
        '''

        PS_flow, W_flow, PS_P2O5, W_P2O5, PS_SO4, W_SO4 = self.plant_input_data
        eff = x[0]
        entrain=(slope_process-C[0]-C[2]*eff-C[3]*eff**2)/C[1]
        bat=LLC.Battery(stages_num=self.stages_num, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,W_P2O5/71*98,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=eff, convergence=1E-3, entrainment_perc_out=[entrain,0])
        error=abs((Prod_P2O5-bat.xout_list[0][1]/98*71)/Prod_P2O5)
        # print (eff, entrain, bat.yout_list[-1][1]/98*71, bat.xout_list[0][1]/98*71, error, bat.runs, ',')
        print ('.', end=' ')
        return error

    def __prepare_summary_table(self, bat, plant_data_table):
        '''
        Prepare a summary table comparing plant data with model data.

        Parameters:
        bat (LLC.Battery): Optimized battery object
        plant_data_table (pandas.DataFrame): Table containing plant data

        Returns:
        pandas.DataFrame: Summary table with plant and model data
        '''

        summary_table = plant_data_table.copy()
        summary_table['Model data']=[bat.Oin_list[0],
                                        bat.yin_list[0][1]/98*71,
                                        bat.Oout_list[-1],
                                        bat.yout_list[-1][1]/98*71,
                                        bat.Ain_list[-1],
                                        bat.Aout_list[0],
                                        bat.xout_list[0][1]/98*71,
                                        f"{bat.Aout_list[0]*bat.xout_list[0][1]/(bat.Oin_list[0]*bat.yin_list[0][1]+bat.Ain_list[-1]*bat.xin_list[-1][1]):.1%}",
                                        f"{bat.stages_num:.0f}",
                                        bat.eff_list[0,1],
                                        str(bat.EQUIL)[17:-2]]
        return summary_table

# Washing 6 stages

In [55]:
PS_P2O5=11.6  # %P2O5 in purified silvent
PS_SO4=0.07  # %SO4 in purified solvent
WS_P2O5=0.2  # %P2O5 in washed solvent
W_flow=23.4  # Water flow
W_P2O5=0  # %P2O5 in water
W_SO4=0 # %SO4 in water. 0 or 0.02
Prod_flow=48.4 # =36.7*1.32 Product acid flow
Prod_P2O5=37.6  # %P2O5 in product acid
plant_data = [PS_P2O5, PS_SO4, WS_P2O5, W_flow, W_P2O5, W_SO4, Prod_flow, Prod_P2O5]

In [56]:
options={
    'method':'secant',
    'damping':.1,
    'max_iter':100,
    'battery_convergence':1e-4,
    'stage_convergence':1e-4
}
# opt_bat=Optimal_Washing_Battery(plant_data, stages_num=6)
opt_bat=Optimal_Washing_CounterCurrentBattery(plant_data, stages_num=6, **options)

slope_process: 0.30319148936170215
Calculating entrainment = f(slope,eff) parameters
. . . . . . 

Calculating optimal eff and entrain
. . . . . 
 optimal data:
 eff: 0.75 entrain: 3.6410591117778397 convergence: 0.0006536150466546091
Calculating entrainment optimal battery simulation
Ready


In [57]:
opt_bat.bat.errors_array.plot()

In [58]:
opt_bat.summary_table

,Plant data,Model data
PS_flow,159.196491,159.196491
PS_P2O5,11.6,11.6
WS_flow,134.196491,129.796968
WS_P2O5,0.2,0.224419
W_flow,23.4,23.4
Prod_flow,48.4,48.073216
Prod_P2O5,37.6,37.555451
P2O5_recovery,98.5%,97.8%
Stages_num,6,6
Efficiency,---,0.75


In [59]:
opt_bat.profile_table

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6
"Org in, ton/hr",159.196491,148.719418,141.586746,134.892578,132.216776,130.969783
"y1 in, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",16.011268,11.017252,7.224803,3.313245,1.595686,0.761176
"y3 in, %",0.070000,0.087732,0.070593,0.040137,0.025263,0.019432
"Aq in, ton/hr",43.011004,35.619409,28.680617,25.907853,24.615529,23.400000
"x1 in, %",1.272676,1.272968,1.273527,1.505210,1.722118,0.000000
"x2 in, %",43.293787,33.383306,19.308977,9.527115,3.575805,0.000000
"x3 in, %",0.273407,0.255997,0.152897,0.069090,0.023894,0.000000
"Org out, ton/hr",148.719321,141.586898,134.891825,132.217256,130.969863,129.796968
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000


In [60]:
opt_bat.bat.yout_list[-1]

array([12.        ,  0.30976151,  0.01674457])

In [61]:
opt_bat.profile_table.to_csv('Profile_table.csv')

## Operation Curve

In [62]:
# @markdown Profile data
data=CopyPasteToArray('''
x	y
0	0.0531
3.01	0.26
10.8	1.22
20.69	6.02
33.12	7.77
34.9	8.86
35.81	10.3
''')
profile_data1=pd.DataFrame(data[1:], columns=data[0])
profile_data1.to_csv('Profile_data.csv')

In [63]:
## P2O5
bat=opt_bat.bat
fig = go.Figure()
n=1 #H3PO4
fig.add_trace(go.Scatter(x=[bat.xin_list[-1][n]/98*71,bat.xout_list[0][n]/98*71], y=[bat.yout_list[-1][n]/98*71,bat.yin_list[0][n]/98*71], name='Plant data', marker_color='orange'))
fig.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig.update_layout(title='', width=1200, height=600,)
fig.show()

## Equilibrium data

In [21]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org	water_org-original
0.011	0.0932	0.072	8.69	1.70	0.0186	12.09
10.325	0.0017	0.229	10.66	1.52	0.02	10.66
10.380	0.0990	0.269	8.31	1.50	0.0208	10.41
19.821	0.0031	0.635	11.3	1.34	0.01884	11.3
19.710	0.0928	0.773	10.36	1.32	0.0195	12.34
28.710	0.0057	1.435	11.96	1.28	0.01978	11.96
27.882	0.0858	1.325	9.7	1.34	0.0222	11.1
35.611	0.0071	2.871	11.94	1.28	0.036	11.94
35.887	0.0846	2.981	12.02	1.24	0.0374	12.6
43.065	0.0082	5.328	12.12	1.28	0.057	12.12
43.065	0.0802	5.604	10.38	1.06	0.0608	11.7
49.966	0.0074	8.723	12.28	1.28	0.0872	12.28
0.017	0.1540	0.042	11.48	1.46	0.0164	13.28
4.831	0.0020	0.146	12.7	1.78	0.01302	12.6
4.941	0.1078	0.173	12.06	1.66	0.0139	11.84
19.600	0.0033	0.828	11.84	1.38	0.01688	11.42
19.766	0.1120	0.856	12.46	1.38	0.01832	11.2
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data3=pd.DataFrame(data[1:], columns=data[0])
# data3_names=CopyPasteToArray('''
# wp12	wp21	wp22	wp31	wp32	wp41	wp42	wp51	wp52	wp61	wp62	wp71	wp13	ww11	ww12	ww21	ww22
# ''')

In [22]:
data3.to_csv('Lab_data.csv')

In [23]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [24]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [25]:
mesh_predict_x=np.array([EQUIL_rotem_washing3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [26]:
fig_equil10_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=opt_bat.bat
# H3PO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil10_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil10_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil10_1.update_scenes(camera_projection_type="orthographic")

fig_equil10_1.show()

In [64]:
entr_p, entr_c = opt_bat.bat.entrainment_perc_list[-1], opt_bat.bat.entrainment_comp_list[-1]

# Raw washing calc

### 10 - Washing5 - optimize.direct()

Plant data

In [68]:
# Overall washing data
PS_P2O5=11.6  # %P2O5 in purified silvent
PS_SO4=0.07  # %SO4 in purified solvent
WS_P2O5=0.2  # %P2O5 in washed solvent
W_flow=23.4  # Water flow
W_P2O5=0  # %P2O5 in water
W_SO4=0  # %SO4 in water. 0 or 0.02
Prod_flow=48.4 # =36.7*1.32 Product acid flow
Prod_P2O5=37.6  # %P2O5 in product acid

D = 5
# initials
OP = 1 # %P2O5
AF = W_flow - D
AP = 3 # %P2O5
AS = 0.02 # %SO4
# data_5_stages = [PS_P2O5, PS_SO4, OP, AF, AP, AS, Prod_flow, Prod_P2O5]

In [69]:
WS_P2O5=OP
W_flow=AF
W_P2O5=AP
W_SO4=AS

A=np.array([[PS_P2O5,-WS_P2O5],[1,-1]])
b=np.array([Prod_flow*Prod_P2O5-W_flow*W_P2O5,Prod_flow-W_flow])
PS_flow, WS_flow=np.linalg.solve(A,b)
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5+W_flow*W_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

Purified solvent flow = 163.6 ton/hr,   Washed solvent flow = 133.6 ton/hr   P2O5 recovery = 93.2%


In [70]:
summary_table10=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",5,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table10

,Plant data
PS_flow,163.645283
PS_P2O5,11.6
WS_flow,133.645283
WS_P2O5,1
W_flow,18.4
Prod_flow,48.4
Prod_P2O5,37.6
P2O5_recovery,93.2%
Stages_num,5
Efficiency,---


#### slope(entr,eff)

נמצא שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
מרובע. מחשבים פרמטרים של פונקציה זאת

slope = a + b * entr + c * eff + d * eff^2

בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]

האופטימיזציה הופכת לאחת עם משתנה בודד.

In [75]:
## P2O5
summary_table=summary_table10
fig10_1 = go.Figure()
n=1 #H3PO4

for entr in [-2, 0, 2]:
  for eff in [.66,.76,.86]:
    bat=LLC.Battery(stages_num=5, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,W_P2O5/71*98,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    print ('entr:', entr, '  eff:', eff, '  slope:', (bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1]))
    fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          # text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='entr:'+str(entr)+', '+'eff:'+str(eff)))

fig10_1.add_trace(go.Scatter(x=[W_P2O5,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))

# bat=LLC.Battery(stages_num=5, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,W_P2O5/71*98,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=.77, convergence=1E-3, entrainment_perc_out=[3.6,0])
# fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
#                           text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
#                           textposition ='bottom center', name='Model data', marker_color='green'))
# fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
# fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

entr: -2   eff: 0.66   slope: 0.3170138472525713
entr: -2   eff: 0.76   slope: 0.3226649901674738
entr: -2   eff: 0.86   slope: 0.325833565181629
entr: 0   eff: 0.66   slope: 0.2953956469509376
entr: 0   eff: 0.76   slope: 0.3012073463229662
entr: 0   eff: 0.86   slope: 0.3061301538182477
entr: 2   eff: 0.66   slope: 0.277373595460128
entr: 2   eff: 0.76   slope: 0.28275750113062925
100 iterations!
entr: 2   eff: 0.86   slope: 0.28673063643505703


In [77]:
slopes=[]
for entr in [-2, 0, 2]: # לינארי -- מספיק 2
  for eff in [.66,.76,.86]: # פורבולה -- צריך 3
    bat=LLC.Battery(stages_num=5, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,W_P2O5,W_SO4], EQUIL=EQUIL_rotem_washing3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    slope=(bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1]) # שיפוע של קו הפעולה
    slopes.append([entr, eff, slope])
    # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
arr=np.array(slopes)
arr

array([[-2.        ,  0.66      ,  0.31366442],
       [-2.        ,  0.76      ,  0.31882981],
       [-2.        ,  0.86      ,  0.32226964],
       [ 0.        ,  0.66      ,  0.29287305],
       [ 0.        ,  0.76      ,  0.29859461],
       [ 0.        ,  0.86      ,  0.30315899],
       [ 2.        ,  0.66      ,  0.27485926],
       [ 2.        ,  0.76      ,  0.2800576 ],
       [ 2.        ,  0.86      ,  0.28398227]])

In [78]:
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=arr[:,2], mode = "markers"))
fig.show()

!!!!!!!!!!!!!!!!!!!!!!!!!!!

הקשר בין סחיפות, יעילות ושיפוע הוא ריבועי ביחס לשניהם. לא נראה לי שהוא ליניארי עבור שיפוע. נראה שצריך לתקן את החישוב.

צריך להוסיף גם עמודה של יעילות בריבוע, ולפתור מטריצה למציאת 5 מקדמים: חופשי, יעילות ויעילות בריבוע, שיפוע ושיפוע בריבוע.

In [ ]:
arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]]
arr_mod

array([[1.        , 3.6       , 0.77      , 0.5929    , 0.30342971],
       [1.        , 3.6       , 0.67      , 0.4489    , 0.30059448],
       [1.        , 3.6       , 0.87      , 0.7569    , 0.30442349],
       [1.        , 2.6       , 0.77      , 0.5929    , 0.3113692 ],
       [1.        , 2.6       , 0.67      , 0.4489    , 0.30925362],
       [1.        , 2.6       , 0.87      , 0.7569    , 0.3123726 ],
       [1.        , 4.6       , 0.77      , 0.5929    , 0.29504003],
       [1.        , 4.6       , 0.67      , 0.4489    , 0.29234033],
       [1.        , 4.6       , 0.87      , 0.7569    , 0.29627614]])

In [ ]:
rows=[0,1,2,3]
A=arr_mod[rows,0:4]
B=arr_mod[rows,4]
C=np.dot(np.linalg.inv(A),B) # מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות
C

array([ 0.26268041, -0.00793949,  0.16093662, -0.09207243])

In [ ]:
pred=np.dot(arr_mod[:,0:4],C)
pred

array([0.30342971, 0.30059448, 0.30442349, 0.3113692 , 0.30853397,
       0.31236298, 0.29549022, 0.29265499, 0.29648401])

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=arr[:,2], mode = "markers"))
# fig.add_trace(go.Scatter3d(x=arr[rows,0], y=arr[rows,1], z=pred[rows], mode = "markers"))
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=pred, mode = "markers"))

fig.show()

#### together

Plant data

In [ ]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6
PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

slope_proc=(PS_P2O5-WS_P2O5)/Prod_P2O5

Purified solvent flow = 159.2 ton/hr,   Washed solvent flow = 134.2 ton/hr   P2O5 recovery = 98.5%


In [ ]:
summary_table10_1=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table10_1

,Plant data
PS_flow,159.196491
PS_P2O5,11.6
WS_flow,134.196491
WS_P2O5,0.2
W_flow,23.4
Prod_flow,48.4
Prod_P2O5,37.6
P2O5_recovery,98.5%
Stages_num,6
Efficiency,---


Slope(entrain,eff)

נמצא שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
מרובע. מחשבים פרמטרים של פונקציה זאת

slope = a + b * entr + c * eff + d * eff^2

בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]

האופטימיזציה הופכת לאחת עם משתנה בודד.

In [ ]:
slopes=[]
entr_initial, eff_init = 3.6, 0.77
for entr in [entr_initial, entr_initial*.75]: # לינארי -- מספיק 2
  for eff in [eff_init,eff_init*.85,eff_init*1.1]: # פורבולה -- צריך 3
    bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    slope=(bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1])
    slopes.append([entr, eff, slope])
    # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
arr=np.array(slopes)
arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]] # entrainment approximated to porabola

In [ ]:
rows=[0,1,2,3] # 4 rows for 4 unknowns
A=arr_mod[rows,0:4]
B=arr_mod[rows,4]
C=np.dot(np.linalg.inv(A),B) # פתרון מערכת משוואות.  מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות

optimize

optimization with 1 parameter

במקום להריץ אופטימיזציה עם 2 משתנים: יעילות וסחיפות, זה רץ על פרמטר בודד -- יעילות. הסחיפות מחושבות מתוך תלות בין סחיפות ושיפוע קו פעולה. תלות זאת חושבה לפני זה.

In [ ]:
def P2O5_calc3_1(x):
  eff = x[0]
  entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]
  bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entrain,0])
  # error1=abs((WS_P2O5-bat.yout[1]/98*71)/WS_P2O5)
  error=abs((Prod_P2O5-bat.xout[1]/98*71)/Prod_P2O5)
  print (eff, entrain, bat.yout[1]/98*71, bat.xout[1]/98*71, error, bat.runs, ',')

  return error

In [ ]:
bounds=[(0.5,1)]
sol10_1=optimize.direct(P2O5_calc3_1, bounds=bounds, f_min=0, f_min_rtol=0.001, maxiter=100, locally_biased=False)
print (sol10_1.x, sol10_1.fun)

0.75 3.58232070311904 0.2316886366651532 37.53650385122883 0.0016887273609354266 40 ,
0.9166666666666666 3.7053610501815397 0.08588284141633287 37.88802595712421 0.0076602648171333 44 ,
0.5833333333333334 2.826956587995283 0.5529074478085162 36.53031845398312 0.028448977287682996 28 ,
0.8055555555555556 3.693592348591124 0.16624116668017097 37.72651921382196 0.003364872708030785 54 ,
0.6944444444444444 3.400790861195705 0.31045687166658387 37.20466528750617 0.010514221076963688 44 ,
0.9722222222222222 3.605858106299873 0.05825211702407979 37.830550460897186 0.006131661194074055 44 ,
0.861111111111111 3.7346057976119584 0.12033207394474967 37.8100317097481 0.005585949727343113 46 ,
0.7685185185185185 3.627217717882095 0.2084089109646816 37.616825219171986 0.0004474792332974582 42 ,
0.7314814814814815 3.529617222083623 0.25810388159773345 37.46173865645521 0.0036771633921486913 38 ,
[0.76851852] 0.0004474792332974582


In [ ]:
eff10_1=sol10_1.x[0]
entrain10_1=entrain=(slope_proc-C[0]-C[2]*eff10_1-C[3]*eff10_1**2)/C[1]

optimal calcul

In [ ]:
bat10_1=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff10_1, convergence=1E-3, entrainment_perc_out=[entrain10_1,0])
bat10_1.runs

42

In [ ]:
bat=bat10_1
summary_table10_1['Model data']=[bat.Oin,bat.yin[1]/98*71,bat.Oout,bat.yout[1]/98*71,bat.Ain,bat.Aout,bat.xout[1]/98*71,f"{bat.Aout*bat.xout[1]/(bat.Oin*bat.yin[1]):.1%}", f"{bat.stages_num:.0f}", bat.eff_list[0,1], str(bat.EQUIL)[17:-2]]
summary_table10_1

,Plant data,Model data
PS_flow,159.196491,159.196491
PS_P2O5,11.6,11.6
WS_flow,134.196491,129.757894
WS_P2O5,0.2,0.208409
W_flow,23.4,23.4
Prod_flow,48.4,48.126303
Prod_P2O5,37.6,37.616825
P2O5_recovery,98.5%,98.0%
Stages_num,6,6
Efficiency,---,0.768519


In [ ]:
table10_1=LLC.BatteryTable(bat10_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6
"Org in, ton/hr",159.196491,148.520452,141.340745,134.532371,132.074524,130.870701
"y1 in, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",16.011268,10.915566,7.085996,3.090851,1.488594,0.704182
"y3 in, %",0.070000,0.088433,0.070263,0.038597,0.024596,0.019185
"Aq in, ton/hr",42.848001,35.404694,28.384919,25.758296,24.557021,23.400000
"x1 in, %",1.272653,1.272953,1.273526,1.494723,1.698311,0.000000
"x2 in, %",43.214779,33.078769,18.620093,9.006942,3.355499,0.000000
"x3 in, %",0.287008,0.268671,0.163577,0.082557,0.041054,0.020000
"Org out, ton/hr",148.530658,141.315647,134.531021,132.032818,130.908907,129.757894
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000


#### vizual

In [ ]:
## P2O5
bat=bat10_1
summary_table=summary_table10_1
fig10_1 = go.Figure()
n=1 #H3PO4
fig10_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig10_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

In [ ]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [ ]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [ ]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [ ]:
fig_equil10_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat10_1
# H3PO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil10_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil10_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil10_1.update_scenes(camera_projection_type="orthographic")

fig_equil10_1.show()

# Washing 1+5 stages

In [ ]:
# Overall washing data
PS_P2O5=11.6  # %P2O5 in purified silvent
PS_SO4=0.07  # %SO4 in purified solvent
WS_P2O5=0.2  # %P2O5 in washed solvent
W_flow=23.4  # Water flow
W_P2O5=0  # %P2O5 in water
W_SO4=0  # %SO4 in water. 0 or 0.02
Prod_flow=48.4 # =36.7*1.32 Product acid flow
Prod_P2O5=37.6  # %P2O5 in product acid
# plant_data = [PS_P2O5, PS_SO4, WS_P2O5, W_flow, W_P2O5, W_SO4, Prod_flow, Prod_P2O5]

## manual 5 stages

In [30]:
options={
    'method':'secant',
    'damping':.1,
    'max_iter':100,
    'battery_convergence':1e-4,
    'stage_convergence':1e-4
}

In [31]:
D = 5
# initials
OP = .5 # %P2O5
AF = W_flow - D
AP = 3 # %P2O5
AS = 0.02 # %SO4
data_5_stages = [PS_P2O5, PS_SO4, OP, AF, AP, AS, Prod_flow, Prod_P2O5]

In [32]:
# A=np.array([[PS_P2O5,-OP],[1,-1]])
# b=np.array([Prod_flow*Prod_P2O5-AF*AP,Prod_flow-AF])
# PS_flow, WS_flow=np.linalg.solve(A,b)
# man_bat=LLC.Battery(stages_num=5, Oin=PS_flow, Ain=AF, yin=[12,PS_P2O5/71*98,PS_SO4], xin=[0,AP/71*98,AS], EQUIL=EQUIL_rotem_washing3, eff=1, convergence=1E-3, entrainment_perc_out=[0,0])
# LLC.BatteryTable(man_bat)

In [34]:
opt_bat=Optimal_Washing_CounterCurrentBattery(data_5_stages, stages_num=5, **options)

slope_process: 0.3208092485549133
Calculating entrainment = f(slope,eff) parameters
. . . . . . 

Calculating optimal eff and entrain
. . . . . . . . . . 100 iterations!
. . . . . 
 optimal data:
 eff: 0.8549382716049382 entrain: -1.6670695136735347 convergence: 0.0008994422696259075
Calculating entrainment optimal battery simulation
Ready


In [35]:
opt_bat.summary_table

,Plant data,Model data
PS_flow,157.625225,157.625225
PS_P2O5,11.6,11.6
WS_flow,127.625225,128.698897
WS_P2O5,0.5,0.265982
W_flow,18.4,18.4
Prod_flow,48.4,49.473771
Prod_P2O5,37.6,37.58068
P2O5_recovery,96.6%,98.7%
Stages_num,5,5
Efficiency,---,0.854938


In [ ]:
opt_bat.profile_table

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",157.625225,144.809418,135.672504,130.548587,129.227161
"y1 in, %",12.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",16.011268,9.734420,4.597609,1.365302,0.478010
"y3 in, %",0.070000,0.094780,0.053552,0.022530,0.017299
"Aq in, ton/hr",34.468635,25.453315,20.401975,19.098662,18.400000
"x1 in, %",1.272655,1.273332,1.550213,1.681128,0.000000
"x2 in, %",39.481630,23.766020,9.312863,5.195032,4.140845
"x3 in, %",0.338997,0.195407,0.045460,0.025528,0.020000
"Org out, ton/hr",144.807387,135.672611,130.550805,129.227653,128.518811
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000


In [ ]:
OF = opt_bat.bat.Oout_list[-1]
OP = opt_bat.bat.yout_list[-1][1]
Oy = opt_bat.bat.yout_list[-1]
entr_p = opt_bat.bat.entrainment_perc_list[-1]
entr_c = opt_bat.bat.entrainment_comp_list[-1]
eff = opt_bat.bat.eff
print ('OF=', OF)
print ('OP=', OP)
print ('Oy=', Oy)
print ('entr_p=', entr_p)
print ('entr_c=', entr_c)
print ('eff=', eff)

OF= 128.5844371824876
OP= 0.2888583257688634
Oy= [12.          0.28885833  0.01650013]
entr_p= [-1.37583227  0.         -1.37583227  0.        ]
entr_c= [[ 1.48650341 10.53787145  0.0660386 ]
 [ 0.          0.          0.        ]]
eff= 0.8796296296296295


In [53]:
## P2O5
bat=opt_bat.bat
fig = go.Figure()
n=1 #H3PO4
fig.add_trace(go.Scatter(x=[bat.xin_list[-1][n]/98*71,bat.xout_list[0][n]/98*71], y=[bat.yout_list[-1][n]/98*71,bat.yin_list[0][n]/98*71], name='Plant data', marker_color='orange'))
fig.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig.update_layout(title='', width=1200, height=600,)
fig.show()

In [54]:
fig.show()

In [ ]:
OF = opt_bat.bat.Oout_list[-1]
OF

129.7404692155328

In [ ]:
OP = opt_bat.bat.yout_list[-1][1]
OP

0.27187106704344705

In [ ]:
opt_bat.bat.entrainment_perc_list[-1]

array([3.64162241, 0.        , 3.64162241, 0.        ])

In [ ]:
opt_bat.bat.entrainment_comp_list[-1]

array([[1.52478972, 8.99793673, 0.0648971 ],
       [0.        , 0.        , 0.        ]])

## optimize.root

In [ ]:
def opt_fun1(x):
    OP, AF, AP, AS = x
    data_5_stages = [PS_P2O5, PS_SO4, OP, AF, AP, AS, Prod_flow, Prod_P2O5]
    opt_bat=Optimal_Washing_CounterCurrentBattery(data_5_stages, stages_num=5, **options)
    # opt_bat=Optimal_Washing_Battery(plant_data, stages_num=6)
    print ('opt_bat')
    OF = opt_bat.bat.Oout_list[-1]
    OP = opt_bat.bat.yout_list[-1][1]
    Oy = opt_bat.bat.yout_list[-1]
    entr_p = opt_bat.bat.entrainment_perc_list[-1]
    entr_c = opt_bat.bat.entrainment_comp_list[-1]
    eff = opt_bat.bat.eff
    opt_stage=LLC.Stage(OF, W_flow, Oy,[0,W_P2O5,W_SO4], EQUIL_rotem_washing3, eff=eff, entrainment_perc=entr_p, entrainment_comp_in=entr_c)
    print ('opt_stage')
    e1 = WS_P2O5 - opt_stage.yout[1]
    e2 = AF + D - opt_stage.Aout
    e3 = AP - opt_stage.xout[1]
    e4 = AS - opt_stage.xout[2]
    print (e1, e2, e3, e4)
    return [e1, e2, e3, e4]

In [ ]:
# sol1 = optimize.root(opt_fun1, [OP, AF, AP, AS], tol=.1)

Calculating entrainment = f(slope,eff) parameters
. . . . . . 

Calculating optimal eff and entrain
. . 100 iterations!
. . . . . . . . 100 iterations!
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 100 iterations!
. 100 iterations!
. . . . . . . . . . . 100 iterations!
. 100 iterations!
. . 100 iterations!
. . . . . . . . . . . . . 100 iterations!
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
 optimal data:
 eff: 0.9992176930270716 entrain: -2.0867981419280075 convergence: 0.020630518847529918
Calculating entrainment optimal battery simulation
30 iterations!
Ready
opt_bat
opt_stage
0.17143298826491837 -0.6526097167437719 2.953995991094074 0.017938654005087085
Calculating entrainment = f(slope,eff) parameters
. . . . . . 

Calculating optimal eff and entrain
. . 100 iterations!
. . .

/content/LLC.py:124: RuntimeWarning:

divide by zero encountered in divide

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning:

invalid value encountered in reduce



SystemError: <built-in function direct> returned a result with an exception set

In [ ]:
sol1.x

NameError: name 'sol1' is not defined

In [ ]:
last=LLC.Stage(30, 15, [12,11.6,0.07],[0,0,0], EQUIL_rotem_washing3, eff=0.8, entrainment_perc=entr_p, entrainment_comp_in=entr_c)
last.roundoutput()

Oout= 26.6
yout= [12.0, 2.896, 0.029]  %water_org, %solubles
yout_tag= [12.0, 0.72, 0.018]  %water_org_equilib, %solubles_equilib
Aout= 18.5
xout= [1.384, 14.416, 0.072]  %solvent_aq, %solubles
equilibrium efficiency= [1.  0.8 0.8]
error= [5.6e-06, -8e-07, -7e-07]
convergence= [7.5e-06, 4.8e-06, 4.3e-06]


In [ ]:
last.entrainment_out

[array([3.64162241, 0.        ]),
 array([12.        ,  2.89604499,  0.02872293]),
 array([ 1.38448814, 14.41646662,  0.07221709])]

In [ ]:
opt_bat=Optimal_Washing_Battery(plant_data, stages_num=5)

In [ ]:
opt_bat.summary_table

In [ ]:
opt_bat.profile_table

In [ ]:
opt_bat.profile_table

In [ ]:
# opt_bat.profile_table.to_csv('Profile_table.csv')

## Operation Curve

In [ ]:
# @markdown Profile data
data=CopyPasteToArray('''
x	y
0	0.0531
3.01	0.26
10.8	1.22
20.69	6.02
33.12	7.77
34.9	8.86
35.81	10.3
''')
profile_data1=pd.DataFrame(data[1:], columns=data[0])
# profile_data1.to_csv('Profile_data.csv')

In [ ]:
## P2O5
bat=opt_bat.bat
summary_table=opt_bat.summary_table
fig10_1 = go.Figure()
n=1 #H3PO4
fig10_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig10_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

## Equilibrium data

In [ ]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org	water_org-original
0.011	0.0932	0.072	8.69	1.70	0.0186	12.09
10.325	0.0017	0.229	10.66	1.52	0.02	10.66
10.380	0.0990	0.269	8.31	1.50	0.0208	10.41
19.821	0.0031	0.635	11.3	1.34	0.01884	11.3
19.710	0.0928	0.773	10.36	1.32	0.0195	12.34
28.710	0.0057	1.435	11.96	1.28	0.01978	11.96
27.882	0.0858	1.325	9.7	1.34	0.0222	11.1
35.611	0.0071	2.871	11.94	1.28	0.036	11.94
35.887	0.0846	2.981	12.02	1.24	0.0374	12.6
43.065	0.0082	5.328	12.12	1.28	0.057	12.12
43.065	0.0802	5.604	10.38	1.06	0.0608	11.7
49.966	0.0074	8.723	12.28	1.28	0.0872	12.28
0.017	0.1540	0.042	11.48	1.46	0.0164	13.28
4.831	0.0020	0.146	12.7	1.78	0.01302	12.6
4.941	0.1078	0.173	12.06	1.66	0.0139	11.84
19.600	0.0033	0.828	11.84	1.38	0.01688	11.42
19.766	0.1120	0.856	12.46	1.38	0.01832	11.2
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data3=pd.DataFrame(data[1:], columns=data[0])
# data3_names=CopyPasteToArray('''
# wp12	wp21	wp22	wp31	wp32	wp41	wp42	wp51	wp52	wp61	wp62	wp71	wp13	ww11	ww12	ww21	ww22
# ''')

In [ ]:
# data3.to_csv('Lab_data.csv')

In [ ]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [ ]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [ ]:
mesh_predict_x=np.array([EQUIL_rotem_washing3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [ ]:
fig_equil = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=opt_bat.bat
# H3PO4_org
fig_equil.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil.update_scenes(camera_projection_type="orthographic")

fig_equil.show()